In [80]:
import time
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
import numpy as np

In [81]:
import pickle
with open('df_sector.pickle','rb') as f:
    df_sector = pickle.load(f)
with open('sector.pickle','rb') as f:
    sector = pickle.load(f)

In [82]:
for sector_name, sector_df in df_sector.items():
    for stock_code, df in sector_df.items():
        df.set_index('일자',inplace=True)

In [83]:
ex = df_sector['Food']['097950']
k = ex.loc['2022-07-22':'2022-08-01',['종가']].values
print(k)
np.mean(k)
np.mean(k)-ex.loc[['2022-07-22'],['종가']].values

[[397500]
 [395000]
 [397000]
 [395000]
 [395000]
 [391500]
 [387000]]


array([[-3500.]])

In [84]:
for sector_name, sector_df in df_sector.items():
    for stock_code, df in tqdm(sector_df.items()):
        df['Label'] = '--'
        for i in range(9,len(df)-1):
            today_close= df.iloc[[i+1],[4]].values
            window_close_mean = np.mean(df.iloc[i:i+10,[4]].values)
            result = today_close-window_close_mean
            if bool(result < -200):
                df['Label'][i+1] = '하향'
            elif bool(result > 200):
                df['Label'][i+1] = '상향'
            else :
                df['Label'][i+1] = '횡보'
        df.drop(df.index[0:10],inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.87s/it]


In [85]:
df_sector['Food']['097950']

,거래량,시가,고가,저가,종가,Label
일자,,,,,,
2010-06-22,62670,217500,218000,212500,214000,하향
2010-06-23,39410,214000,216000,213000,215000,하향
2010-06-24,41070,215000,218500,213500,218000,하향
2010-06-25,83149,220000,221000,217500,220000,하향
2010-06-28,65542,221000,223500,220500,223000,하향
...,...,...,...,...,...,...
2022-07-27,16778,396500,400000,394000,395000,상향
2022-07-28,25944,395500,398000,391000,395000,상향
2022-07-29,27618,397500,397500,389000,391500,상향


In [44]:
'''
df = df_sector['Food']['097950']

df['Label'] = '--'
for i in range(9,len(df)-1):
    today_close= df.iloc[[i+1],[4]].values
    window_close_mean = np.mean(df.iloc[i:i+10,[4]].values)
    result = today_close-window_close_mean
    if bool(result < -200):
        df['Label'][i+1] = '하향'
    elif bool(result > 200):
        df['Label'][i+1] = '상향'
    else :
        df['Label'][i+1] = '횡보'
'''

In [54]:
df['Label'].value_counts()

하향    1404
상향    1381
횡보     205
Name: Label, dtype: int64

In [104]:
label_ratio_dic = {}
for sector_name, sector_df in df_sector.items():
    label_ratio_dic[sector_name] = {}
    for stock_code, df in sector_df.items():
        label_ratio = df['Label'].value_counts(normalize=True)
        print(label_ratio)
        label_ratio_dic[sector_name][stock_code] = label_ratio
        
with open('label_ratio_dic.pickle', 'wb') as f:
    pickle.dump(label_ratio_dic,f)


상향    0.490301
하향    0.487960
횡보    0.021739
Name: Label, dtype: float64
하향    0.472918
상향    0.460792
횡보    0.066289
Name: Label, dtype: float64
상향    0.376923
하향    0.340803
횡보    0.282274
Name: Label, dtype: float64
상향    0.49097
하향    0.47893
횡보    0.03010
Name: Label, dtype: float64
상향    0.470234
하향    0.468562
횡보    0.061204
Name: Label, dtype: float64
상향    0.376254
하향    0.351171
횡보    0.272575
Name: Label, dtype: float64
하향    0.407692
상향    0.395652
횡보    0.196656
Name: Label, dtype: float64
하향    0.361873
상향    0.352843
횡보    0.285284
Name: Label, dtype: float64
횡보    0.593311
하향    0.232107
상향    0.174582
Name: Label, dtype: float64
하향    0.512709
상향    0.473244
횡보    0.014047
Name: Label, dtype: float64
하향    0.480602
상향    0.478930
횡보    0.040468
Name: Label, dtype: float64
상향    0.468896
하향    0.454515
횡보    0.076589
Name: Label, dtype: float64
하향    0.532441
상향    0.458528
횡보    0.009030
Name: Label, dtype: float64
하향    0.486622
상향    0.472241
횡보    0.041137
Name: Lab

In [ ]:
'''for sector_name, sector_df in df_sector.items():
    for stock_code, df in sector_df.items():
        if df['Label'].value_counts(normalize=True)['횡보']>0.2 :
            del df
            sector[sector_name].remove(stock_code)
'''

# 2. 데이터 분할& 원-핫인코딩

In [56]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train_dic = {}
X_test_dic={}
y_train_dic = {}
y_test_dic = {}
#모델이 각 업종마다 존재한다. 18개
for sector_name, sector_dic in df_sector.items():
    X_train_dic[sector_name] = {}
    X_test_dic[sector_name]={}
    y_train_dic[sector_name] = {}
    y_test_dic[sector_name] = {}
    
    for stock_code, df in sector_dic.items():
        # 단 셔플 없이 분할
        X_train, X_test, y_train, y_test = train_test_split(df.drop('Label', 1), df['Label'], test_size=0.2, random_state=0, shuffle=False)
        X_train_dic[sector_name][stock_code] = X_train
        X_test_dic[sector_name][stock_code] = X_test
        y_train = pd.get_dummies(y_train)
        y_train_dic[sector_name][stock_code] = y_train
        y_test = pd.get_dummies(y_test)
        y_test_dic[sector_name][stock_code] = y_test

In [57]:
for sector_name, stock_code_list in sector.items():
    for stock_code in stock_code_list:
        print(stock_code)
        print('훈련 데이터셋 : ', X_train_dic[sector_name][stock_code].shape, y_train_dic[sector_name][stock_code].shape)
        print('테스트 데이터셋 : ', X_test_dic[sector_name][stock_code].shape, y_test_dic[sector_name][stock_code].shape)
        print('-'*40)

097950
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
271560
훈련 데이터셋 :  (989, 5) (989, 3)
테스트 데이터셋 :  (248, 5) (248, 3)
----------------------------------------
000080
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
004370
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
005300
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
093050
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
020000
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
105630
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
001070
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
051910
훈련 데이터셋 :  (23

In [58]:
y_train_dic['Food']['097950']

,상향,하향,횡보
일자,,,
2010-06-22,0,1,0
2010-06-23,0,1,0
2010-06-24,0,1,0
2010-06-25,0,1,0
2010-06-28,0,1,0
...,...,...,...
2020-02-26,1,0,0
2020-02-27,1,0,0
2020-02-28,0,1,0


# 스케일링 하기

In [59]:
from sklearn.preprocessing import MinMaxScaler

X_train_scaled_dic = {}
X_test_scaled_dic = {}
#y_train_scaled_dic = {}
#y_test_scaled_dic = {}
X_scaler_dic = {}
#y_scaler_dic = {}
#스케일을 적용할 column을 정의합니다.\
#스케일을 훈련 데이터 세트로 fit()을 수행한 결과를 이용해 테스트 데이터셋을 변환
#X_train_dic -> X_scaled_train_dic / X_test_dic -> X_scaled_test_dic
scale_cols = ['거래량','시가','고가', '저가','종가']
for sector_name, sector_dic in X_train_dic.items():
    X_train_scaled_dic[sector_name] = {}
    X_test_scaled_dic[sector_name] = {}
    #y_train_scaled_dic[sector_name] = {}
    #y_test_scaled_dic[sector_name] = {}
    X_scaler_dic[sector_name] = {}
    #y_scaler_dic[sector_name] = {}

    for stock_code, df in sector_dic.items():
        X_scaler = MinMaxScaler()
        #훈련데이터를 정규화 학습 및 변환
        X_train_scaled = X_scaler.fit_transform(df[scale_cols])
        X_train_scaled_dic[sector_name][stock_code] = pd.DataFrame(X_train_scaled,index = df.index,columns=scale_cols)
        #테스트 데이터를 훈련데이터 scaler로 정규화
        test_df = X_test_dic[sector_name][stock_code]
        X_test_scaled = X_scaler.transform(test_df[scale_cols])
        X_test_scaled_dic[sector_name][stock_code] = pd.DataFrame(X_test_scaled,index = test_df.index,columns=scale_cols)
        X_scaler_dic[sector_name][stock_code] = X_scaler

In [60]:
for sector_name, stock_code_list in sector.items():
    for stock_code in stock_code_list:
        print(stock_code)
        print('훈련 데이터셋 : ', X_train_scaled_dic[sector_name][stock_code].shape, y_train_dic[sector_name][stock_code].shape)
        print('테스트 데이터셋 : ', X_test_scaled_dic[sector_name][stock_code].shape, y_test_dic[sector_name][stock_code].shape)
        print('-'*40)

097950
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
271560
훈련 데이터셋 :  (989, 5) (989, 3)
테스트 데이터셋 :  (248, 5) (248, 3)
----------------------------------------
000080
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
004370
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
005300
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
093050
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
020000
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
105630
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
001070
훈련 데이터셋 :  (2392, 5) (2392, 3)
테스트 데이터셋 :  (598, 5) (598, 3)
----------------------------------------
051910
훈련 데이터셋 :  (23

In [61]:
with open('X_train_scaled_dic.pickle','wb') as f:
    pickle.dump(X_train_scaled_dic, f)
with open('X_test_scaled_dic.pickle','wb') as f:
    pickle.dump(X_test_scaled_dic, f)
with open('y_train_dic.pickle','wb') as f:
    pickle.dump(y_train_dic, f)
with open('y_test_dic.pickle','wb') as f:
    pickle.dump(y_test_dic, f)

In [62]:
with open('X_scaler_dic.pickle', 'wb') as f:
    pickle.dump(X_scaler_dic,f)